In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as ww
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.ngetnews.com/news/articleList.html?page={}&total=67933&view_type=sm&box_idxno=0"
article_base = "https://www.ngetnews.com/"

dict = defaultdict(list)
for page in range(1, 300):
    url = base_url.format(page)
    driver.get(url)
    time.sleep(2)

    # 기사 제목과 링크 추출

    ul_elem = driver.find_element(By.CSS_SELECTOR, 'ul.type2')
    li_list = ul_elem.find_elements(By.TAG_NAME, 'li')
    print(li_list)
    for li in li_list:
        a_tag = li.find_element(By.CSS_SELECTOR, 'a')
        href = a_tag.get_attribute('href')
        web_id = href.split('=')[1]

        article_url = 'https://www.ngetnews.com/news/articleView.html?idxno=' + web_id

        # open new window with article page
        driver.execute_script('window.open(arguments[0]);', article_url)
        driver.switch_to.window(driver.window_handles[1])

        article_soup = bs(driver.page_source, 'html.parser')

        dict['Category'].append(article_soup.select_one('#articleViewCon > article > header > p > a').get_text(strip=True))
        dict['Title'].append(article_soup.select_one('#articleViewCon > article > header > h1').get_text(strip=True))
        dict['Date'].append(article_soup.select_one('#articleViewCon > article > header > ul > li:nth-child(2)').get_text(strip=True))


        sentences = []

        atc = article_soup.select_one('#article-view-content-div')
        p_tags = atc.find_all('p')

        for p_tag in p_tags:
            sentences.append(p_tag.get_text(strip = True))

        dict['Contents'].append(' '.join(sentences))


        driver.close()
        driver.switch_to.window(driver.window_handles[0])
driver.quit()

print(dict)

In [ ]:
with open('articles_300.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4) 